In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, unix_timestamp, udf, lag, sum as spark_sum, avg, when, count
from pyspark.sql.window import Window
from pyspark.sql.types import StringType, DoubleType
from shapely.geometry import Point, shape
import json

spark = (SparkSession.builder
                    .appName('NewYorkCityTaxiData')
                    .getOrCreate()
        )

In [2]:
spark

In [3]:
nyc_taxi_data = spark.read.option("header", "true").csv("input/Sample NYC Data.csv")

nyc_taxi_data.show()

+--------------------+--------------------+---------+---------+------------------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+
|           medallion|        hack_license|vendor_id|rate_code|store_and_fwd_flag|pickup_datetime|dropoff_datetime|passenger_count|pickup_longitude|pickup_latitude|dropoff_longitude|dropoff_latitude|
+--------------------+--------------------+---------+---------+------------------+---------------+----------------+---------------+----------------+---------------+-----------------+----------------+
|89D227B655E5C82AE...|BA96DE419E711691B...|      CMT|        1|                 N| 01-01-13 15:11|  01-01-13 15:18|              4|      -73.978165|      40.757977|       -73.989838|       40.751171|
|0BD7C8F5BA12B88E0...|9FD8F69F0804BDB55...|      CMT|        1|                 N| 06-01-13 00:18|  06-01-13 00:22|              1|      -74.006683|      40.731781|       -73.994499|        40.75066|


In [ ]:
nyc_taxi_data = nyc_taxi_data.withColumn("pickup_longitude", col("pickup_longitude").cast(DoubleType()))
nyc_taxi_data = nyc_taxi_data.withColumn("pickup_latitude", col("pickup_latitude").cast(DoubleType()))
nyc_taxi_data = nyc_taxi_data.withColumn("dropoff_longitude", col("dropoff_longitude").cast(DoubleType()))
nyc_taxi_data = nyc_taxi_data.withColumn("dropoff_latitude", col("dropoff_latitude").cast(DoubleType()))


In [24]:
# Load Borough GeoJSON Data
with open("input/nyc-boroughs.geojson") as f:
    geojson_data = json.load(f)
    # for feature in geojson_data["features"]:
    #     print(feature["properties"]["borough"])

# Broadcast GeoJSON Data
geo_broadcast = spark.sparkContext.broadcast(geojson_data)

In [26]:
def get_borough(lon, lat):
    if lon is None or lat is None:
        return None
    point = Point(lon, lat)
    for feature in geo_broadcast.value["features"]:
        polygon = shape(feature["geometry"])
        if polygon.contains(point):
            return feature["properties"]["borough"]
    return "Unknown"

borough_udf = udf(get_borough, StringType())

<function get_borough at 0x7f5530267880>


In [27]:
nyc_taxi_data = nyc_taxi_data.withColumn("pickup_borough", borough_udf("pickup_longitude", "pickup_latitude"))
nyc_taxi_data = nyc_taxi_data.withColumn("dropoff_borough", borough_udf("dropoff_longitude", "dropoff_latitude"))


In [28]:
# Convert Datetime Columns
nyc_taxi_data = nyc_taxi_data.withColumn("pickup_ts", unix_timestamp(col("pickup_datetime"), "dd-MM-yy HH:mm"))
nyc_taxi_data = nyc_taxi_data.withColumn("dropoff_ts", unix_timestamp(col("dropoff_datetime"), "dd-MM-yy HH:mm"))

# Calculate Trip Duration
nyc_taxi_data = nyc_taxi_data.withColumn("duration", col("dropoff_ts") - col("pickup_ts"))

# Filter Negative & Unrealistic Trips (more than 4 hours)
nyc_taxi_data = nyc_taxi_data.filter((col("duration") > 0) & (col("duration") <= 4 * 3600))


In [29]:
# Window to Partition by Taxi ID and Order by Pickup Time
window_spec = Window.partitionBy("medallion").orderBy("pickup_ts")

# Previous Dropoff Time
nyc_taxi_data = nyc_taxi_data.withColumn("previous_dropoff", lag("dropoff_ts").over(window_spec))

# Idle Time Calculation
nyc_taxi_data = nyc_taxi_data.withColumn("idle_time", col("pickup_ts") - col("previous_dropoff"))

# Filter Outliers (Idle Time <= 4 hours)
nyc_taxi_data = nyc_taxi_data.filter((col("idle_time") > 0) & (col("idle_time") <= 4 * 3600))

# Aggregate Utilization
utilization_df = nyc_taxi_data.groupBy("medallion").agg(
    spark_sum("duration").alias("total_trip_time"),
    spark_sum("idle_time").alias("total_idle_time")
)

# Utilization Formula
utilization_df = utilization_df.withColumn("utilization", col("total_trip_time") / (col("total_trip_time") + col("total_idle_time")))

print("Utilization per Taxi:")
utilization_df.show(truncate=False)

Utilization per Taxi:
+--------------------------------+---------------+---------------+-------------------+
|medallion                       |total_trip_time|total_idle_time|utilization        |
+--------------------------------+---------------+---------------+-------------------+
|000318C2E3E6381580E5C99910A60668|12960          |17400          |0.4268774703557312 |
|002E3B405B6ABEA23B6305D3766140F1|7980           |16140          |0.3308457711442786 |
|0030AD2648D81EE87796445DB61FCF20|1440           |720            |0.6666666666666666 |
|0036961468659D0BFC7241D92E8ED865|10260          |19740          |0.342              |
|0038EF45118925A510975FD0CCD67192|9840           |15120          |0.3942307692307692 |
|0053334C798EC6C8E637657962030F99|7500           |22440          |0.250501002004008  |
|005DED7D6E6C45441C26981DCFBED992|10740          |11760          |0.47733333333333333|
|005F00B38F46E2100F4A5D0B6AAAD3AC|17880          |42180          |0.2977022977022977 |
|00790C7BAD30B7A9E096

In [37]:
# Search Time Calculation

nyc_taxi_data = nyc_taxi_data.withColumn("search_time", col("pickup_ts") - col("previous_dropoff"))

# Filter Outliers for Search Time
nyc_taxi_data = nyc_taxi_data.filter((col("search_time") > 0) & (col("search_time") <= 4 * 3600))

# Average Search Time per Borough
avg_search_df = nyc_taxi_data.groupBy("dropoff_borough").agg(avg("search_time").alias("avg_search_time"))

print("Average Search Time per Borough:")
# avg_search_df.show(truncate=False)

nyc_taxi_data.select("previous_dropoff", "pickup_ts", "search_time").show(10)


Average Search Time per Borough:


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [40]:
# Same Borough Trips
# same_borough_df = nyc_taxi_data.filter(col("pickup_borough") == col("dropoff_borough")) \
#     .groupBy("pickup_borough").count() \
#     .withColumnRenamed("count", "same_borough_trips")

# print("Same Borough Trips:")
# same_borough_df.show(truncate=False)

# Trips that started and ended within the same borough
same_borough_trips_df = nyc_taxi_data.filter(col("pickup_borough") == col("dropoff_borough"))

# Count the trips per borough
same_borough_count_df = same_borough_trips_df.groupBy("pickup_borough").agg(count("medallion").alias("trip_count"))

print("Number of trips that started and ended within the same borough:")
same_borough_count_df.show(truncate=False)


Number of trips that started and ended within the same borough:


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/socket.py", line 718, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
# Different Borough Trips
diff_borough_df = nyc_taxi_data.filter(col("pickup_borough") != col("dropoff_borough")) \
    .groupBy("pickup_borough", "dropoff_borough").count() \
    .withColumnRenamed("count", "different_borough_trips")

print("Different Borough Trips:")
diff_borough_df.show(truncate=False)